In [1]:
import pandas as pd
import numpy as np
import requests
from pymongo import MongoClient
import sys, getopt
import warnings
import time
import json
from datetime import date
warnings.filterwarnings('ignore')

In [2]:
try:
    opts, args = getopt.getopt(sys.argv[1:], "hf:", ["forum="])
except getopt.GetoptError:
    print('dcard_forum_articles.py -f <forum name>')
    sys.exit(2)
    
for opt, arg in opts:
    if opt == '-h':
        print('dcard_forum_articles.py -f <forum name>')
        sys.exit()
    elif opt in ("-f", "--forum"):
        forum = arg

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [4]:
forum = 'alcohol'

In [5]:
dcard_col = client['Dcard'][forum]

In [6]:
last_time_data = dcard_col.find().sort("updatedAt", -1).limit(1)
for i in last_time_data:
    last_time = i['updatedAt']
last_time

'2021-03-27T05:44:49.034Z'

In [7]:
forums = forum
url = f'https://www.dcard.tw/service/api/v2/forums/{forums}/posts'

In [ ]:
params = {
    'limit':1,
    'popular':'false'
}

In [8]:
params = {
    'limit':1,
    'popular':'false'
}

while True:
    a = requests.get(url, params=params)
    data_list = a.json()
    
    if len(data_list) == 0:
        break

    data_time_list = [i for i in data_list if i['updatedAt'] > last_time]
    
    if len(data_time_list) > 0:
        for i in data_time_list:
            if dcard_col.find_one({'id': i['id']}):
                dcard_col.update_many({'id': i['id']}, {"$set":i}, upsert = True)
            else:
                dcard_col.insert(data_time_list)

    last_data = data_list[-1]
    last_id = last_data['id']
    
    print('post_id：'+str(last_id))
    print('post_title：'+str(last_data['title']))
    print('='*20)
    
    params = {
        'limit':100,
        'popular':'false',
        'before': last_id
    }

post_id：235636715
post_title：調酒師實戰手冊(II):12種雞尾酒中常見基本風味特徵與口感
post_id：235287482
post_title：過年品飲
post_id：235004980
post_title：【品酒】金標響17
post_id：234710848
post_title：「台北酒吧分享」流年當鋪-當吧
post_id：234304237
post_title：兔子酒
post_id：233918229
post_title：[品酒文］The Drink Malt Kawaii Animal Series Tormore 19Yo
post_id：233628150
post_title：[心得] 工藝琴酒全書
post_id：233252201
post_title：[威士忌] 雙獅會 x Or Sileis 高原騎士2007年單一麥芽威士忌原酒
post_id：232888508
post_title：＃新年威士忌禮盒  布納哈本12年
post_id：232437803
post_title：［調酒］教父🇮🇹
post_id：231978408
post_title：#酒廠製程介紹 Mortlach 2.81次蒸餾
post_id：231781953
post_title：#公告 品酒板板規


## 原版

In [ ]:
while True:
    a = requests.get(url, params=params)
    data = a.json()
    
    if len(data) == 0:
        break
        
#     dcard_col.insert_many(data)
    
    last_data = data[-1]
    last_id = last_data['id']
    
    print('post_id：'+str(last_id))
    print('post_title：'+str(last_data['title']))
    print('='*20)
    
    params = {
        'limit':100,
        'popular':'false',
        'before': last_id
    }
    
    time.sleep(0.5)